In [26]:
import pandas as pd
import numpy as np
from datetime import datetime
from pytz import UTC
import matplotlib.pyplot as plt

SYMBOLS = [ 'AUD_CAD', 'AUD_CHF', 'AUD_JPY', 'AUD_NZD', 'AUD_USD', 
            'CAD_CHF', 'CAD_JPY', 'CHF_JPY', 'EUR_AUD', 'EUR_CAD', 
            'EUR_CHF', 'EUR_GBP', 'EUR_JPY', 'EUR_NZD', 'EUR_USD', 
            'GBP_AUD', 'GBP_CAD', 'GBP_CHF', 'GBP_JPY', 'GBP_NZD', 
            'GBP_USD', 'NZD_CAD', 'NZD_CHF', 'NZD_JPY', 'NZD_USD', 
            'USD_CAD', 'USD_CHF', 'USD_JPY'
          ]


df = pd.read_csv("../../db/data/merge/secondary/logs_.csv", index_col=0)
df_primary = pd.read_csv("../../db/data/merge/primary/closes.csv", index_col=0)
df.index = pd.to_datetime(df.index)
df_primary.index = pd.to_datetime(df_primary.index)
datetime(2022, 7, 1, 0, 0, 0, 0, UTC)

df = df.loc[datetime(2010, 6, 1, 0, 0, 0, 0, UTC):datetime(2022, 7, 1, 0, 0, 0, 0, UTC)]
df_primary =  df_primary.loc[datetime(2010, 6, 1, 0, 0, 0, 0, UTC):datetime(2022, 7, 1, 0, 0, 0, 0, UTC)]


In [5]:
def mean_prediction(df, rate=24):
    data = pd.DataFrame([])
    for ccy in df.columns:
        data[ccy] = df[ccy].rolling(rate).mean()
    data.index = df.index
    return data
        
def random_prediction(df):
    data = pd.DataFrame([])
    
    for sym in SYMBOLS:
        data[f'{sym}']=pd.DataFrame(np.random.uniform(low=-0.8,high=0.8,size=(df.shape[0],1)))
    data.index = df.index
    return data

rand = random_prediction(df)
mu = mean_prediction(df)

def condition(x):
    if x>0:
        return 1
    elif x< 0:
        return -1
    else:
        return 0

for sym in mu.columns:
    mu[sym] = mu[sym].map( lambda x: condition(x))
    
# mu.reset_index(inplace=True, drop=True)

display(rand.shape, mu.shape)
clas = mu.iloc[-1:]



(74593, 28)

(74593, 28)

In [152]:
pred_last = rand.iloc[-120:]
pred = rand.iloc[-1:]

pred = abs(pred)
pred




,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
2022-07-01 00:00:00+00:00,0.773086,0.091571,0.304924,0.238711,0.288257,0.342453,0.472317,0.782083,0.389087,0.624956,...,0.203738,0.597739,0.336592,0.788746,0.120724,0.351322,0.472205,0.791942,0.543428,0.414714


In [180]:
def correlation_pairs(df, rate=1):

    data = pd.DataFrame(index=df.index)
    for pair in df.columns:
        for second_pair in df.columns:
            data[f'{pair}_{second_pair}_corr'] = df[pair].corr(df[second_pair])
            # for the rolling correlation if we manage to do it...
            # data[f'{pair}_{second_pair}_corr'] = df[pair].rolling(rate).corr(df[second_pair]) 
    data.index = df.index
    data = data.iloc[-1:].T
    data.columns = ["correlation"]
    data = data[round(data["correlation"],6) != 1.000000]
    data = data.T
    
    return data

def weighted_volatility(volatility):
    weight = pd.DataFrame([])
    volatility = volatility.copy()
    volatility_T = volatility.T
    weight = volatility_T/ volatility_T.mean()
    return  weight

In [182]:
c = correlation_pairs(pred_last)
c

/tmp/ipykernel_3975/3687875360.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{pair}_{second_pair}_corr'] = df[pair].corr(df[second_pair])
/tmp/ipykernel_3975/3687875360.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{pair}_{second_pair}_corr'] = df[pair].corr(df[second_pair])
/tmp/ipykernel_3975/3687875360.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

,AUD_CAD_AUD_CHF_corr,AUD_CAD_AUD_JPY_corr,AUD_CAD_AUD_NZD_corr,AUD_CAD_AUD_USD_corr,AUD_CAD_CAD_CHF_corr,AUD_CAD_CAD_JPY_corr,AUD_CAD_CHF_JPY_corr,AUD_CAD_EUR_AUD_corr,AUD_CAD_EUR_CAD_corr,AUD_CAD_EUR_CHF_corr,...,USD_JPY_GBP_CHF_corr,USD_JPY_GBP_JPY_corr,USD_JPY_GBP_NZD_corr,USD_JPY_GBP_USD_corr,USD_JPY_NZD_CAD_corr,USD_JPY_NZD_CHF_corr,USD_JPY_NZD_JPY_corr,USD_JPY_NZD_USD_corr,USD_JPY_USD_CAD_corr,USD_JPY_USD_CHF_corr
correlation,0.151206,0.085416,0.045353,0.05693,-0.094568,0.022202,-0.143998,0.114067,-0.006306,0.070283,...,-0.114528,-0.022393,0.06461,0.175127,0.077792,0.021798,0.013684,0.03997,-0.083603,0.010896


In [183]:
display(c, pred_last.iloc[-1:])

,AUD_CAD_AUD_CHF_corr,AUD_CAD_AUD_JPY_corr,AUD_CAD_AUD_NZD_corr,AUD_CAD_AUD_USD_corr,AUD_CAD_CAD_CHF_corr,AUD_CAD_CAD_JPY_corr,AUD_CAD_CHF_JPY_corr,AUD_CAD_EUR_AUD_corr,AUD_CAD_EUR_CAD_corr,AUD_CAD_EUR_CHF_corr,...,USD_JPY_GBP_CHF_corr,USD_JPY_GBP_JPY_corr,USD_JPY_GBP_NZD_corr,USD_JPY_GBP_USD_corr,USD_JPY_NZD_CAD_corr,USD_JPY_NZD_CHF_corr,USD_JPY_NZD_JPY_corr,USD_JPY_NZD_USD_corr,USD_JPY_USD_CAD_corr,USD_JPY_USD_CHF_corr
correlation,0.151206,0.085416,0.045353,0.05693,-0.094568,0.022202,-0.143998,0.114067,-0.006306,0.070283,...,-0.114528,-0.022393,0.06461,0.175127,0.077792,0.021798,0.013684,0.03997,-0.083603,0.010896


,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
2022-07-01 00:00:00+00:00,0.773086,0.091571,-0.304924,0.238711,-0.288257,-0.342453,0.472317,-0.782083,0.389087,0.624956,...,0.203738,-0.597739,-0.336592,-0.788746,0.120724,-0.351322,-0.472205,0.791942,-0.543428,-0.414714


In [10]:
sym = 'AUD_JPY'
sym2 = 'AUD_CAD'

c.filter(regex=sym+'_'+sym2).values[0][0]

0.08541620271727127

In [169]:
def expected_volatility(pred_vol, correlations):
    
    vols = pred_vol.copy()
    pred_vol = abs(pred_vol)
    
    
    for sym in pred_vol.columns:
        
        real_vol = 0
        for sym2 in pred_vol.columns:
            
            if sym == sym2:
                real_vol += pred_vol[sym]
                
            else:
                real_vol += pred_vol[sym2] * correlations.filter(regex=sym+'_'+sym2).values[0][0]
                
        vols[sym] = abs(round(real_vol.mean(),6))
        
    total_vol = vols.mean(axis=1)[0]
    

    return total_vol, vols
    
        


In [170]:
t, v = expected_volatility(pred,c)


print('Total volatilty:', t, '\nReal vols:\n', v)

,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
2022-07-01 00:00:00+00:00,0.773086,0.091571,0.304924,0.238711,0.288257,0.342453,0.472317,0.782083,0.389087,0.624956,...,0.203738,0.597739,0.336592,0.788746,0.120724,0.351322,0.472205,0.791942,0.543428,0.414714


,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
2022-07-01 00:00:00+00:00,0.761466,0.618152,0.188485,0.580053,0.091988,0.390204,0.062418,0.536577,0.75072,0.945439,...,0.699669,0.693661,0.322315,0.841223,0.003483,0.046189,0.070572,1.285203,0.444617,0.34291


Total volatilty: 0.4810262142857143 
Real vols:
                             AUD_CAD   AUD_CHF   AUD_JPY   AUD_NZD   AUD_USD  \
2022-07-01 00:00:00+00:00  0.761466  0.618152  0.188485  0.580053  0.091988   

                            CAD_CHF   CAD_JPY   CHF_JPY  EUR_AUD   EUR_CAD  \
2022-07-01 00:00:00+00:00  0.390204  0.062418  0.536577  0.75072  0.945439   

                           ...   GBP_JPY   GBP_NZD   GBP_USD   NZD_CAD  \
2022-07-01 00:00:00+00:00  ...  0.699669  0.693661  0.322315  0.841223   

                            NZD_CHF   NZD_JPY   NZD_USD   USD_CAD   USD_CHF  \
2022-07-01 00:00:00+00:00  0.003483  0.046189  0.070572  1.285203  0.444617   

                           USD_JPY  
2022-07-01 00:00:00+00:00  0.34291  

[1 rows x 28 columns]


In [121]:
clas["AUD_CAD"] = 0
clas
t
display(v, pred)


/tmp/ipykernel_3975/3520224859.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clas["AUD_CAD"] = 0


,2022-07-01 00:00:00+00:00
USD_JPY,0.4147


,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
2022-07-01 00:00:00+00:00,0.773086,0.091571,0.304924,0.238711,0.288257,0.342453,0.472317,0.782083,0.389087,0.624956,...,0.203738,0.597739,0.336592,0.788746,0.120724,0.351322,0.472205,0.791942,0.543428,0.414714


In [215]:
def trade_signals(pred_vols,last_data, classification, exchange_rate, risk=0.01, balance=100000, leverage=1):
    cor = correlation_pairs(last_data)
    
    t,v = expected_volatility(pred_vols, cor)
    n = int((abs(classification.T)).sum())
    risked = risk*balance*leverage/(t*n)
 
    
    trade_signals = []
    for ccy in classification.columns:
        d = {}
        if int(classification[ccy]) != 0:
            d["currency"] = ccy
            d["side"] = "sell" if int(classification[ccy]) == -1 else "buy"
            if ccy.split("_")[0] == "USD":
                d["size"] = round(risked)
            else:
                d["size"] = round(risked*float(exchange_rate[ccy.split("_")[0]]))
            d["datetime"] = v.index.to_list()[0].strftime("%Y-%m-%dT%H:%M:%S.Z")
            trade_signals.append(d)
    
    return trade_signals

In [211]:
def read_exchange_rate(df):
    
    rate = df.iloc[-1:]
    
    data = pd.DataFrame(index=rate.index)
    
    for ccy in rate.columns:
        if ccy.split("_",2)[0] == "USD":
            data[(ccy.split("_")[1])] = rate[ccy]
        elif ccy.split("_",2)[1] == "USD":
            data[(ccy.split("_")[0])] = 1/rate[ccy]
        
    return data
            
    

In [190]:
er = read_exchange_rate(df_primary)
er

,AUD,EUR,GBP,NZD,CAD,CHF,JPY
2022-07-01 00:00:00+00:00,1.451168,0.954982,0.822774,1.604776,1.28781,0.9556,135.962


In [216]:
ex = trade_signals(pred,pred_last, clas, er)
ex

/tmp/ipykernel_3975/3687875360.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{pair}_{second_pair}_corr'] = df[pair].corr(df[second_pair])
/tmp/ipykernel_3975/3687875360.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{pair}_{second_pair}_corr'] = df[pair].corr(df[second_pair])
/tmp/ipykernel_3975/3687875360.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
2022-07-01 00:00:00+00:00,0.773086,0.091571,0.304924,0.238711,0.288257,0.342453,0.472317,0.782083,0.389087,0.624956,...,0.203738,0.597739,0.336592,0.788746,0.120724,0.351322,0.472205,0.791942,0.543428,0.414714


,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
2022-07-01 00:00:00+00:00,0.761466,0.618152,0.188485,0.580053,0.091988,0.390204,0.062418,0.536577,0.75072,0.945439,...,0.699669,0.693661,0.322315,0.841223,0.003483,0.046189,0.070572,1.285203,0.444617,0.34291


[{'currency': 'AUD_CHF',
  'side': 'buy',
  'size': 112,
  'datetime': '2022-07-01T00:00:00.Z'},
 {'currency': 'AUD_JPY',
  'side': 'sell',
  'size': 112,
  'datetime': '2022-07-01T00:00:00.Z'},
 {'currency': 'AUD_NZD',
  'side': 'sell',
  'size': 112,
  'datetime': '2022-07-01T00:00:00.Z'},
 {'currency': 'AUD_USD',
  'side': 'buy',
  'size': 112,
  'datetime': '2022-07-01T00:00:00.Z'},
 {'currency': 'CAD_CHF',
  'side': 'buy',
  'size': 99,
  'datetime': '2022-07-01T00:00:00.Z'},
 {'currency': 'CAD_JPY',
  'side': 'sell',
  'size': 99,
  'datetime': '2022-07-01T00:00:00.Z'},
 {'currency': 'CHF_JPY',
  'side': 'sell',
  'size': 74,
  'datetime': '2022-07-01T00:00:00.Z'},
 {'currency': 'EUR_AUD',
  'side': 'sell',
  'size': 74,
  'datetime': '2022-07-01T00:00:00.Z'},
 {'currency': 'EUR_CAD',
  'side': 'buy',
  'size': 74,
  'datetime': '2022-07-01T00:00:00.Z'},
 {'currency': 'EUR_CHF',
  'side': 'buy',
  'size': 74,
  'datetime': '2022-07-01T00:00:00.Z'},
 {'currency': 'EUR_GBP',
  'sid